# ML HW5
### 106598018 萬俊瑋

**1. Show the following is true. If 𝐽 = −{𝑑log2 𝑦 + (1 − 𝑑)log2(1 − 𝑦)} and y = 𝑓(𝐱) = 1/(1+exp(−𝒘𝑇𝐱)), then 𝜕𝐽/𝜕𝑤𝑗 = 𝑥𝑗(𝑦 − 𝑑).**

![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q1.jpg?raw=true)

**2. Compute the complete update (back propagation) equations for all weights (𝑤1~𝑤8) in the following neural networks. This time, the activation function in the hidden layer is ReLU and that in the output layer remains sigmoid. In addition, use softmax as the cost function.**
![](https://github.com/wei840222/ML-HW/blob/master/images/HW4Q4NN.JPG?raw=true)
![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q2.jpg?raw=true)

**3. Write a program to implement the neural network with your back propagation equations in problem 2. To test your network, train it to distinguish the classes of versicolor and virginica in the Iris dataset using only the third and fourth features (i.e., petal length and petal width) as the inputs. As usual, use 70% of the data for training and the rest for testing. Repeat the experiments 10 times to find the average accuracy. During training, set the desired output as 1.0 for in class data and 0.0 for out of class data. Don’t forget to use random numbers as the initial weights. In addition, monitor the cost (loss) function with respect to the training epochs. Do you observe any difference when compared with the cost function plot using MSE?**

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Loading the dataset
dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
dataset = dataset[50:150]
dataset = pd.get_dummies(dataset, columns=[4]) # One Hot Encoding
values = list(dataset.columns.values)

y = dataset[values[-2:]]
y = np.array(y, dtype='float32')
X = dataset[values[2:4]]
X = np.array(X, dtype='float32')

# Shuffle Data
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]
y_values = y[indices]

# Creating a Train and a Test Dataset
test_size = 30
X_test = X_values[-test_size:]
X_train = X_values[:-test_size]
y_test = y_values[-test_size:]
y_train = y_values[:-test_size]

repeat = 10
accuracyList = []

for i in range(repeat):
    seed = np.random.randint(0, 9999)
    np.random.seed(seed)
    tf.set_random_seed(seed)

    # Session
    sess = tf.Session()

    # Interval / Epochs
    interval = 100
    epoch = 1000

    # Initialize placeholders
    X_data = tf.placeholder(shape=[None, 2], dtype=tf.float32)
    y_target = tf.placeholder(shape=[None, 2], dtype=tf.float32)

    # Input neurons : 2
    # Hidden neurons : 2
    # Output neurons : 2
    hidden_layer_nodes = 2

    # Create variables for Neural Network layers
    w1 = tf.Variable(tf.random_normal(shape=[2,hidden_layer_nodes])) # Inputs -> Hidden Layer
    b1 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes]))   # First Bias
    w2 = tf.Variable(tf.random_normal(shape=[hidden_layer_nodes,2])) # Hidden layer -> Outputs
    b2 = tf.Variable(tf.random_normal(shape=[2]))   # Second Bias

    # Operations
    hidden_output = tf.nn.relu(tf.add(tf.matmul(X_data, w1), b1))
    final_output = tf.nn.softmax(tf.add(tf.matmul(hidden_output, w2), b2))

    # Cost Function
    loss = tf.reduce_mean(-tf.reduce_sum(y_target * tf.log(final_output), axis=1))

    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

    # Initialize variables
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training
    print('Training the model...')
    for i in range(1, (epoch + 1)):
        sess.run(optimizer, feed_dict={X_data: X_train, y_target: y_train})
        if i % interval == 0:
            print('Epoch', i, '|', 'Loss:', sess.run(loss, feed_dict={X_data: X_train, y_target: y_train}))

    # Prediction
    accuracy = 0
    for i in range(len(X_test)):
        t = np.array([0., 0.])
        t[np.argmax(y_test[i])] = 1.
        y = np.rint(sess.run(final_output, feed_dict={X_data: [X_test[i]]}))[0]

        if t[0] == y[0] and t[1] == y[1]:
            accuracy += 1
    print('Accuracy:', accuracy/test_size, '\n')
    accuracyList.append(accuracy/test_size)

print('Average Accuracy:', np.mean(accuracyList)) 

Training the model...
Epoch 100 | Loss: 0.58835024
Epoch 200 | Loss: 0.49967963
Epoch 300 | Loss: 0.44239414
Epoch 400 | Loss: 0.40417913
Epoch 500 | Loss: 0.3758228
Epoch 600 | Loss: 0.35276833
Epoch 700 | Loss: 0.33308285
Epoch 800 | Loss: 0.31591016
Epoch 900 | Loss: 0.30075023
Epoch 1000 | Loss: 0.28728387
Accuracy: 0.9333333333333333 

Training the model...
Epoch 100 | Loss: 0.92699987
Epoch 200 | Loss: 0.7615071
Epoch 300 | Loss: 0.67207664
Epoch 400 | Loss: 0.62015074
Epoch 500 | Loss: 0.5841241
Epoch 600 | Loss: 0.5533642
Epoch 700 | Loss: 0.52509093
Epoch 800 | Loss: 0.4980234
Epoch 900 | Loss: 0.47205558
Epoch 1000 | Loss: 0.4473364
Accuracy: 0.7666666666666667 

Training the model...
Epoch 100 | Loss: 0.4498188
Epoch 200 | Loss: 0.41880053
Epoch 300 | Loss: 0.3905035
Epoch 400 | Loss: 0.36500916
Epoch 500 | Loss: 0.34224862
Epoch 600 | Loss: 0.32204795
Epoch 700 | Loss: 0.30417544
Epoch 800 | Loss: 0.28837836
Epoch 900 | Loss: 0.2744082
Epoch 1000 | Loss: 0.26203334
Accuracy

**4. The following shows the LeNet-5 architecture. If we use the weight-sharing approach for the first layer, compute the number of connections and trainable weights from the input layer to the first hidden layer C1. To compute the results, you need the following parameters: kernel size = 5x5, stride = 1, and no zero-padding.**

![](https://github.com/wei840222/ML-HW/blob/master/images/HW5LeNet-5.JPG?raw=true)
![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q4.jpg?raw=true)

**5. Assuming that the following is a part of convolution neural networks. Compute the resultant values (size of 3x3) if it has two input planes, stride of one, no zero-padding, and using the ReLU activation function.**

![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q5.jpg?raw=true)